## Importing Libraries

In [106]:
import numpy as np
from math import sqrt
import pandas as pd

## Reading files

In [107]:
movies_df = pd.read_csv(r'C:\Users\saeid\OneDrive\Desktop\movies.csv')
ratings_df = pd.read_csv(r'C:\Users\saeid\OneDrive\Desktop\ratings.csv')

## Content_Based Recommender


## We want to determine score for each genre according to user ratings and assign score for each movie and recommend them to the user.

## This is a set of movies that the user has watched and rated them

In [108]:
userInput = [
            {'title':'Father of the Bride Part II', 'rating':4},
            {'title':'No Game No Life: Zero', 'rating':3},
            {'title':'Grumpier Old Men', 'rating':5},
            {'title':"Sudden Death", 'rating':4.5},
            {'title':'Heat', 'rating':5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Father of the Bride Part II,4.0
1,No Game No Life: Zero,3.0
2,Grumpier Old Men,5.0
3,Sudden Death,4.5
4,Heat,5.0


In [109]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Now, We have to convert the movies dataframe to our desired form to work on it.

In [110]:
movies_df['Year']=movies_df['title'].str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['Year']=movies_df['Year'].str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df.title.str.strip()
movies_df['genres']=movies_df['genres'].str.split('|')
movies_df.head()

C:\Users\saeid\AppData\Local\Temp\ipykernel_5816\3750492074.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,Year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


## Finding movie ID and adding to user rating.

In [111]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies=pd.merge(inputMovies,inputId)
inputMovies = inputMovies.drop(columns=['genres','Year'])
inputMovies.head()

,title,rating,movieId
0,Father of the Bride Part II,4.0,5
1,No Game No Life: Zero,3.0,193583
2,Grumpier Old Men,5.0,3
3,Sudden Death,4.5,9
4,Heat,5.0,6


In [112]:
inputMovies['rating']

0    4.0
1    3.0
2    5.0
3    4.5
4    5.0
Name: rating, dtype: float64

## generating a dataframe that includes movies with genres, in cofluence 1 and otherwise 0.

In [113]:
moviesWithGenres_df = movies_df.copy()
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,Year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## generating movies with genres for user movies

In [114]:
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,Year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,Heat,"[Action, Crime, Thriller]",1995,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,Sudden Death,[Action],1995,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9738,193583,No Game No Life: Zero,"[Animation, Comedy, Fantasy]",2017,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
serMovies = userMovies.reset_index(drop=True)
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('Year', 1)
userGenreTable=np.asanyarray(userGenreTable)
userGenreTable

C:\Users\saeid\AppData\Local\Temp\ipykernel_5816\3713285694.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('Year', 1)
C:\Users\saeid\AppData\Local\Temp\ipykernel_5816\3713285694.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('Year', 1)
C:\Users\saeid\AppData\Local\Temp\ipykernel_5816\3713285694.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('Year', 1)
C:\Users\saeid\AppData\Local\Temp\ipykernel_5816\3713285694.py:2: FutureWarning: In a future version of pa

array([[0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

## assigning score for each genre.Multiply the N-by-five matrix by the five-by-one matrix, which becomes the N-by-one matrix.

In [116]:
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

array([ 0. ,  5. ,  0. , 12. ,  5. ,  4. ,  0. ,  9.5,  5. ,  5. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ])

## generating movies with genres for all movies.

In [117]:
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('Year', 1)
genreTable.head()

C:\Users\saeid\AppData\Local\Temp\ipykernel_5816\1989552232.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('Year', 1)
C:\Users\saeid\AppData\Local\Temp\ipykernel_5816\1989552232.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('Year', 1)
C:\Users\saeid\AppData\Local\Temp\ipykernel_5816\1989552232.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('Year', 1)
C:\Users\saeid\AppData\Local\Temp\ipykernel_5816\1989552232.py:2: FutureWarning: In a future version of pandas all arg

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## assigning score for each movie by and making our recommender system by multiplying scores of genres by genres and aggregating  on each movie  to find the best movies.

In [118]:
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.483516
2    0.109890
3    0.351648
4    0.351648
5    0.263736
dtype: float64

In [119]:
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
recommendationTable_df.head()

movieId
4719     0.890110
27328    0.802198
83349    0.802198
31367    0.780220
7007     0.692308
dtype: float64

## The first 20 movies that recommended for intended user.

In [120]:
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,Year
364,420,Beverly Hills Cop III,"[Action, Comedy, Crime, Thriller]",1994
660,861,Supercop (Police Story 3: Supercop) (Jing cha ...,"[Action, Comedy, Crime, Thriller]",1992
666,876,Supercop 2 (Project S) (Chao ji ji hua),"[Action, Comedy, Crime, Thriller]",1993
1399,1918,Lethal Weapon 4,"[Action, Comedy, Crime, Thriller]",1998
3430,4673,Tango & Cash,"[Action, Comedy, Crime, Thriller]",1989
3460,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
4368,6387,Once a Thief (Zong heng si hai),"[Action, Comedy, Crime, Thriller]",1991
4691,7004,Kindergarten Cop,"[Action, Comedy, Crime, Thriller]",1990
4693,7007,"Last Boy Scout, The","[Action, Comedy, Crime, Drama, Thriller]",1991
5379,8968,After the Sunset,"[Action, Adventure, Comedy, Crime, Thriller]",2004


## ----------------------------------------------------------------------------------------------------------------------------------

## Collabortive filtering_UserBased

In [121]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## Timestamp has no importance for our solotion, so is droped from rankings dataframe.

In [122]:
ratings_df=ratings_df.drop('timestamp',1)

C:\Users\saeid\AppData\Local\Temp\ipykernel_5816\2308053707.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings_df=ratings_df.drop('timestamp',1)


In [123]:
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [124]:
inputMovies.head()

,title,rating,movieId
0,Father of the Bride Part II,4.0,5
1,No Game No Life: Zero,3.0,193583
2,Grumpier Old Men,5.0,3
3,Sudden Death,4.5,9
4,Heat,5.0,6


## In order to find the most similar users, we find the users who has watched the most common music with the intended user.

In [125]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubsetGroup = userSubset.groupby(['userId'])
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
userSubsetGroup = userSubsetGroup[0:100]
userSubsetGroup

C:\Users\saeid\AppData\Local\Temp\ipykernel_5816\436543416.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)


[(269,
         userId  movieId  rating
  39032     269        3     4.0
  39033     269        5     3.0
  39034     269        6     5.0
  39035     269        9     2.0),
 (270,
         userId  movieId  rating
  39061     270        3     3.0
  39062     270        5     3.0
  39063     270        6     3.0
  39065     270        9     3.0),
 (337,
         userId  movieId  rating
  51894     337        3     4.0
  51895     337        5     4.0
  51896     337        6     5.0
  51898     337        9     5.0),
 (492,
         userId  movieId  rating
  79297     492        3     4.0
  79298     492        5     3.0
  79299     492        6     3.0
  79301     492        9     5.0),
 (6,
       userId  movieId  rating
  561       6        3     5.0
  563       6        5     5.0
  564       6        6     4.0),
 (68,
         userId  movieId  rating
  10362      68        3     2.0
  10363      68        5     2.0
  10364      68        6     4.0),
 (117,
         userId  movieId  

## Next step is finding a quantity to show similarity between users and the intended user. For this purpose, we use from Pearson corrlation between ratings in the inputMovies that have been scored by the intended user and rankings in the userSubsetGroup that have been scored by the other users.(corr=sxy/(sxx*syy))

In [126]:
pearsonCorrelationDict = {}
for name, group in userSubsetGroup:
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    nRatings = len(group)
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


## Generating a dataframe that includes Similarity and the userId and sorting by similarity desc. we use top 50, because correlation with the amount of not possitive will not help us.

In [127]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF

,similarityIndex,userId
0,0.674200,269
1,0.000000,270
2,0.301511,337
3,0.090909,492
4,-0.500000,6
...,...,...
95,0.000000,297
96,0.000000,302
97,0.000000,305
98,0.000000,307


In [128]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers

,similarityIndex,userId
33,1.000000,288
22,1.000000,45
28,1.000000,121
40,1.000000,451
44,1.000000,561
41,1.000000,474
31,1.000000,179
43,1.000000,521
25,1.000000,84
12,0.981981,437


## Merging above dataframe with rating and movieId for assigning score to movies.

In [129]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating

,similarityIndex,userId,movieId,rating
0,1.0,288,1,4.5
1,1.0,288,2,2.0
2,1.0,288,3,4.0
3,1.0,288,5,2.0
4,1.0,288,10,3.0
...,...,...,...,...
21592,0.0,266,3108,5.0
21593,0.0,266,3113,1.0
21594,0.0,266,3142,1.0
21595,0.0,266,3156,1.0


## Now we can calculate Movies scores by multiplying  Similarity index by rating  and aggregate for each movie, then dividing this amount by sum of similarityIndex for each movie.(each movie is equivalent with group by movieId.)

In [130]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,288,1,4.5,4.5
1,1.0,288,2,2.0,2.0
2,1.0,288,3,4.0,4.0
3,1.0,288,5,2.0,2.0
4,1.0,288,10,3.0,3.0
...,...,...,...,...,...
21592,0.0,266,3108,5.0,0.0
21593,0.0,266,3113,1.0,0.0
21594,0.0,266,3142,1.0,0.0
21595,0.0,266,3156,1.0,0.0


In [131]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating

,sum_similarityIndex,sum_weightedRating
movieId,,
1,12.243582,51.264554
2,7.176962,21.424456
3,6.877557,24.315316
4,1.000000,3.000000
5,14.725563,39.185283
...,...,...
193579,0.000000,0.000000
193581,0.000000,0.000000
193583,0.000000,0.000000


## Generating a dataframe includes the final recommendation score by movieId and sorting by final recommendation score desc.

In [132]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
5650,5.0,5650
4495,5.0,4495
866,5.0,866
891,5.0,891
905,5.0,905
170705,5.0,170705
906,5.0,906
5034,5.0,5034
5137,5.0,5137


## Using recommendation system to recommend the best movies for the intended user.

In [133]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,genres,Year
661,866,Bound,"[Crime, Drama, Romance, Thriller]",1996
675,891,Halloween: The Curse of Michael Myers (Hallowe...,"[Horror, Thriller]",1995
687,905,It Happened One Night,"[Comedy, Romance]",1934
688,906,Gaslight,"[Drama, Thriller]",1944
3320,4495,Crossing Delancey,"[Comedy, Romance]",1988
3660,5034,"Truly, Madly, Deeply","[Drama, Romance]",1991
3719,5137,Scratch,[Documentary],2001
3784,5292,Slap Shot,[Comedy],1977
4003,5650,Strange Brew,[Comedy],1983
9497,170705,Band of Brothers,"[Action, Drama, War]",2001


## Enjoy watching yout favorite movies.